Importing required libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error

Importing Datasets

First one was the output of bert when we passed the scraped through it tweeets.
Second One is the true labels, which we calculted from provided csv  

In [ ]:
sentient_from_bert = pd.read_csv('/content/drive/MyDrive/bert_output.csv',index_col = 0)
true_sentiment = pd.read_csv('/content/drive/MyDrive/from_stocks.csv')

In [13]:
df = pd.read_csv('/content/drive/MyDrive/MC data/stock_data.csv',index_col = 0)

In [14]:
df

,0,1,2,3,4,5,6,7
0,2018-01-02 00:00:00,112.040001,116.110001,112.040001,114.480003,114.480003,187400.0,ICLR
1,2018-01-03 00:00:00,114.489998,114.940002,113.660004,114.019997,114.019997,379800.0,ICLR
2,2018-01-04 00:00:00,114.339996,114.739998,112.650002,113.910004,113.910004,422700.0,ICLR
3,2018-01-05 00:00:00,114.220001,117.980003,113.629997,117.620003,117.620003,299000.0,ICLR
4,2018-01-08 00:00:00,117.669998,117.989998,115.680000,117.370003,117.370003,277000.0,ICLR
...,...,...,...,...,...,...,...,...
219061,2022-12-23 00:00:00,176.610001,177.759995,175.080002,177.360001,176.983261,371800.0,JBHT
219062,2022-12-27 00:00:00,177.850006,178.960007,175.050003,178.220001,177.841446,354400.0,JBHT
219063,2022-12-28 00:00:00,178.389999,179.279999,174.350006,174.460007,174.089432,240200.0,JBHT
219064,2022-12-29 00:00:00,175.119995,178.610001,174.779999,176.770004,176.394516,362400.0,JBHT


Splitting The DataSet and scaling it Into Test And Train


In [15]:
df2 = df[df.iloc[:,7] == 'AMZN'].iloc[:,4]

In [ ]:
df2

In [17]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df2).reshape(-1,1))

In [18]:
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

A function that creates timestams that are to be passed in lstm


In [19]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [20]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [21]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

Models

In [28]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 100, 50)           10400     
                                                                 
 lstm_5 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_6 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=10,batch_size=64,verbose=1)

Epoch 1/10
12/12 [==============================] - 11s 473ms/step - loss: 0.0696 - val_loss: 0.0122
Epoch 2/10
12/12 [==============================] - 3s 224ms/step - loss: 0.0115 - val_loss: 0.0143
Epoch 3/10
12/12 [==============================] - 3s 216ms/step - loss: 0.0053 - val_loss: 0.0081
Epoch 4/10
12/12 [==============================] - 3s 214ms/step - loss: 0.0026 - val_loss: 0.0070
Epoch 5/10
12/12 [==============================] - 3s 218ms/step - loss: 0.0024 - val_loss: 0.0084
Epoch 6/10
12/12 [==============================] - 4s 366ms/step - loss: 0.0021 - val_loss: 0.0072
Epoch 7/10
12/12 [==============================] - 3s 223ms/step - loss: 0.0021 - val_loss: 0.0075
Epoch 8/10
12/12 [==============================] - 3s 218ms/step - loss: 0.0020 - val_loss: 0.0082
Epoch 9/10
12/12 [==============================] - 3s 217ms/step - loss: 0.0020 - val_loss: 0.0068
Epoch 10/10
12/12 [==============================] - 3s 229ms/step - loss: 0.0020 - val_loss: 0.007

Predicting on train and test set


In [31]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

11/11 [==============================] - 0s 44ms/step


De normalising the predictions


In [32]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

RMSE on Test Dataset


In [33]:
math.sqrt(mean_squared_error(y_train,train_predict))

115.0338521777242

Saving trained weights for future 

In [34]:
model.save_weights('/content/drive/MyDrive/lstm_weights.hdf5')